# Import Libraries

In [20]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
import joblib


# Load And Process DATA

In [21]:
# Load the CSV
df = pd.read_csv("groundwater_yearly_averages.csv")

# Drop rows with missing important values
df.dropna(subset=['Station_name', 'lat', 'long', 'Year', 'level'], inplace=True)

# Encode station name
station_encoder = LabelEncoder()
df['Station_ID'] = station_encoder.fit_transform(df['Station_name'])

# Features: station ID, lat, long, year
X = df[['Station_ID', 'lat', 'long', 'Year']].values
y = df['level'].values

# Scale the features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)


# Split the data

In [22]:
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)


# Build the Neural Network Model

In [23]:
model = Sequential([
    Dense(128, activation='relu', input_shape=(X_train.shape[1],)),
    Dense(128, activation='relu'),
    Dense(64, activation='relu'),
    Dense(1)  # Output layer: one continuous value
])

model.compile(optimizer='adam', loss='mse', metrics=['mae'])
history = model.fit(X_train, y_train, validation_split=0.2, epochs=50, batch_size=64)


Epoch 1/50
338/338 [==============================] - 2s 6ms/step - loss: 48.9967 - mae: 3.5896 - val_loss: 100.4046 - val_mae: 3.5718
Epoch 2/50
338/338 [==============================] - 2s 5ms/step - loss: 44.3707 - mae: 3.2518 - val_loss: 98.9617 - val_mae: 3.4658
Epoch 3/50
338/338 [==============================] - 2s 6ms/step - loss: 43.6680 - mae: 3.1849 - val_loss: 98.2951 - val_mae: 3.5307
Epoch 4/50
338/338 [==============================] - 2s 5ms/step - loss: 43.4243 - mae: 3.1655 - val_loss: 98.5773 - val_mae: 3.2870
Epoch 5/50
338/338 [==============================] - 2s 5ms/step - loss: 43.2595 - mae: 3.1382 - val_loss: 98.5854 - val_mae: 3.6073
Epoch 6/50
338/338 [==============================] - 2s 6ms/step - loss: 43.1101 - mae: 3.1415 - val_loss: 98.1657 - val_mae: 3.4008
Epoch 7/50
338/338 [==============================] - 2s 5ms/step - loss: 42.9449 - mae: 3.1167 - val_loss: 99.0856 - val_mae: 3.8788
Epoch 8/50
338/338 [==============================] - 2s 5ms/

# Save Model


In [24]:
model.save("groundwater_level_model.h5")
joblib.dump(station_encoder, "station_encoder.pkl")
joblib.dump(scaler, "scaler.pkl")


['scaler.pkl']

In [ ]:
def predict_ground_level(station_name, year):
    # Load encoders and model
    import tensorflow as tf
    import joblib
    import numpy as np

    station_encoder = joblib.load("station_encoder.pkl")
    scaler = joblib.load("scaler.pkl")
    model = tf.keras.models.load_model("groundwater_level_model.h5")

    # Get lat/long of station
    station_row = df[df['Station_name'] == station_name].iloc[0]
    station_id = station_encoder.transform([station_name])[0]

    lat = station_row['lat']
    long = station_row['long']

    # Prepare input
    input_data = np.array([[station_id, lat, long, year]])
    input_scaled = scaler.transform(input_data)

    # Predict
    predicted_level = model.predict(input_scaled)[0][0]

    # Print all info
    print(f"📍 Station: {station_name}")
    print(f"🧭 Location: Latitude = {lat}, Longitude = {long}")
    print(f"📅 Year: {year}")
    print(f"💧 Predicted Groundwater Level: {predicted_level:.2f} m")

    return predicted_level, lat, long


In [30]:
predict_ground_level("Karondiya", 2026)


1/1 [==============================] - 0s 78ms/step


19.131746